In [41]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
import csv
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from tqdm import tqdm

### Let's do web scrapping of afisha.ru: we need to collect info about restaurants in Moscow

#### Firstly, we'll collect the restaurants links by going through all the pages with Selenium.

In [50]:
base_url = "https://www.afisha.ru"
url = f"{base_url}/msk/restaurants/restaurant_list/"

options = webdriver.ChromeOptions()
# options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

driver.get(url)
time.sleep(7)

all_links = set()  #We need to use set to avoid repeat links

while True:
    soup = BeautifulSoup(driver.page_source, "html.parser")
    for a in soup.find_all("a"):
        if "href" in a.attrs and a["href"].startswith("/msk/restaurant"): #Search only links starts with /msk/restaurant
            all_links.add(a["href"])

    try:
        next_button = driver.find_element(By.CLASS_NAME, 'Pagination_pagination-next__rtqsZ')#try to click next page button
        next_button.click()
        time.sleep(5)
    except Exception as e:
        try:
            close_button = driver.find_element(By.CLASS_NAME, 'popmechanic-close')#unexpectedly a banner pops up on the site, try to close it, as it covers the usual page layout
            close_button.click()
            time.sleep(2)

            next_button = driver.find_element(By.CLASS_NAME, 'Pagination_pagination-next__rtqsZ')#try to click next page button again
            next_button.click()
            time.sleep(5)
        except Exception as inner_e:
            print("There are no more pages:", inner_e)
            break

driver.quit()

In [52]:
#adding base url to links
full_links = [base_url + link for link in all_links]
all_links_list = list(full_links)

df = pd.DataFrame(all_links_list, columns=['Link'])
df.to_csv('resto_links.csv', index=False)

print(f"We saved {len(all_links_list)} links in 'rest_links.csv'")

We saved 11957 links in 'rest_links.csv'


In [23]:
import numpy as np

df = pd.read_csv("resto_links.csv")
names = ["for_dima_1", "for_dima_2", "for_goozel_1", "for_goozel_2", "for_vera_1", "for_vera_2"]
for i, name in enumerate(np.array_split(df, 6)):
    name.to_csv(f"{names[i]}.csv", index=False)

In [ ]:
""" 
1. Запускаем ячейку с библиотеками вверху
2. Запускаем ячейку снизу
3. В следующую ячейку вместо test_links.csv выставляем имя своего файла, например, for_dima_2.csv
4. Разделила по 2 части ссылок на каждого, но можно еще дробить, если будет выполняться слишком долго судя по прогрессу в выводе ячейки
5. Если парсинг по какой-то причине сбросился, ячейка с кодом остановилась: уже спаршеное можно сохранить в другой файл или поставить вывод результата в файл с другим названием(data1, data2...), 
посмотреть по прогресс бару, на какой ссылке остановился код, убрать из читаемого файла все ссылки до этой отметки, и запустить парсинг опять
6. Если экран заблокируется, парсинг сбросится, можно включить какой-нибудь фильм без звука на полный экран, либо что-то делать в компьютере на время выполнения, либо в настройках запретить блокировку экрана
7. Потом все файлы с результатами парсинга можно объединить
8. Возможно, мы не успеем спарсить все, но нужно постараться собрать побольше)))
"""


In [3]:
def get_reviews_count(button):
    content_div = button.find('div', class_='Button_button__content___D2b_')
    if content_div:
        counter_span = content_div.find('span', class_='DefaultTag_button__counter__64UpQ')
        if counter_span:
            return int(counter_span.text)
    return 0

In [ ]:

options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

with open('data4.csv', 'w', encoding='utf-8-sig', newline='') as d:
    file_writer =  csv.writer(d, delimiter = ";")
    file_writer.writerow(["title", "rating", "address", "metro", "avg_check", "breakfast", "business_lunch", "deleviry", "parking", "catering", "banquets", "telephone", "site", "restaurant_type", "open_hours", "positive_reviews", "negative_reviews"]) 

with open('for_vera_2.csv','r') as f:
    all_links = f.readlines()


for i, link in enumerate(tqdm(all_links, desc="Links processing", unit="link", dynamic_ncols=True)):
    try:

        driver.get(link)
        driver.implicitly_wait(10)

        soup = BeautifulSoup(driver.page_source, "html.parser")

            
        name = soup.find('span',class_ = "Title_header__SIloF").text
        rank = soup.find('div',class_ = "RestaurantCover_rating-wrapper__CTNts")
        if rank!=None:
            estimate = rank.text
        else:
            estimate = 0
        adress = soup.find('div', class_="SectionTitle_wrapper__nAAJ0 RestaurantExtraInfo_address__aJsK2").text
        metro = soup.find_all('ul', class_="RestaurantExtraInfo_metro-list__KTBX3")
        metro_=[]
        for i in metro:
            for j in i.find_all('span', class_="Text_text__e9ILn"):
                metro_ += [j.text]
        s = ''       
        all = soup.find_all('div',class_ = "RestaurantExtraInfo_table__l34_J")
        for i in all:
            for j in i.find_all('span', class_="Text_text__e9ILn"):
                s += j.text + ','
            tot =s.split(',')[1:len(s):2]   
        bill = tot[0] 
        brekfast = tot[1]
        business =  tot[2]
        delivery = tot[3]
        parking = tot[4]
        keit = tot[5]
        feast = tot[6] 
        phone_num = tot[7]
        site_ = tot[8] 
        type_all = []
        type_ = soup.find_all('div',class_ = "RestaurantExtraInfo_tag__BqQ7e")
        for i in type_:
            for j in i.find_all('div', class_="Button_button__content___D2b_"):
                type_all +=[j.text] 
        work = soup.find_all('div',class_ = "RestaurantCover_content-wrapper__72Dox")
        for i in work:
            for j in i.find_all('span', class_="Text_text__e9ILn"):
                e = j          
        open_ = e.text.strip('Открыто c')    

        filters_div = soup.find('div', class_='FiltersReview_filters__7E8qs')
        positive_reviews = 0
        negative_reviews = 0

        if filters_div:
            buttons = filters_div.find_all('button', class_='Button_button__j_Rc9')
            for button in buttons:
                content_div = button.find('div', class_='Button_button__content___D2b_')
                if content_div:
                    text = content_div.text
                    if 'Положительные' in text:
                        positive_reviews = get_reviews_count(button)
                    elif 'Отрицательные' in text:
                        negative_reviews = get_reviews_count(button)

                    
        with open('data.csv', 'a', encoding='utf-8-sig', newline='') as d:
            file_writer =  csv.writer(d, delimiter = ";")
            file_writer.writerow([name, estimate, adress, metro_, bill, brekfast ,business ,delivery ,parking ,keit ,feast ,phone_num , site_, type_all, open_, positive_reviews, negative_reviews])

    except TypeError:
        pass
    except Exception as e:
        tqdm.write(f"Link processing error {link}: {e}")
driver.quit()

#### Creating DataFrame
Using CSV-file with whole data set

In [48]:
df = pd.read_csv('big_data.csv', delimiter=';')

#### Fixing rating column

In [ ]:
def fix_rating_column(
        data_frame: pd.DataFrame
) -> pd.DataFrame:
    '''
        There are cells with the pattern "n отзывов", they are replaced by '0'.
        The rating column is converted to the float (numeric) type.
        The average value of the review is set to 0.

        :param data_frame: pd.DataFrame

        :return: pd.DataFrame
    '''
    data_frame.loc[data_frame['rating'].str.contains('отз'), 'rating'] = '0'
    data_frame['rating'] = pd.to_numeric(data_frame['rating'])

    mean_rating = round(data_frame['rating'][data_frame['rating'] != 0.0].mean(), 2)
    data_frame['rating'] = np.where(data_frame['rating'] == 0.0, mean_rating, data_frame['rating'])

    return data_frame


df = fix_rating_column(data_frame=df)
print(df) # if you want to see the result

                    title  rating  \
0        One Price Coffee     6.9   
1                   Rioba     6.9   
2             Мята Lounge     6.9   
3                   Farsh     6.9   
4             Лакшми и Ко     6.9   
...                   ...     ...   
12429             Сабкафе     6.9   
12430  Кофевар Капучинофф     6.9   
12431    One Price Coffee     6.9   
12432            Il marco     6.9   
12433          Papa Grill     6.9   

                                                 address  \
0       Москва, Автозаводская, 18, ТРЦ «Ривьера», 1 этаж   
1      Москва, Маршала Прошлякова, 14, ТЦ Metro Cash ...   
2                                Москва, Костромская, 17   
3      Москва, просп. Вернадского, 86а, ТРЦ Avenue So...   
4                        Москва, Херсонская, 43, корп. 3   
...                                                  ...   
12429  Москва, 13-й км МКАД, 2, АЗС «Нефтьмагистраль ...   
12430                           Москва, Баженова, КПП №8   
12431          

In [ ]:
import pandas as pd
import csv
import os 

df = pd.read_csv('C:/Users/Arthur/YandexDisk/Гузель/магистратура/Python/Git/analysis-project/analysis-project/big_data.csv', sep=';')
station_metro = 'Цветной бульвар,Тверская,Чеховская,БоровицкаяаяПолянкаДобрынинскаяСерпуховскаяМаяковскаяТеатральнаяНовокузнецкаяПавелецкаяПушкинскаяКузнецкий мостКитай-городТаганскаяМарксистскаяСмоленскаяАрбатскаяАлександровский садПлощадь РеволюцииЧкаловскаяКурскаяПарк культурыКропоткинскаяБиблиотека им. ЛенинаОхотный РядЛубянкаКрасные воротаЧистые прудыКрасные воротаОктябрьскаяТретьяковскаяТургеневскаяСухаревскаяПроспект МираТрубнаяСретенский бульвар' 
df['breakfast'] = (df['breakfast'] =='Есть').astype(int)
df['business_lunch'] = (df['business_lunch'] =='Есть').astype(int)
df['deleviry'] = (df['deleviry'] =='Есть').astype(int)
df['parking'] = (df['parking'] =='Есть').astype(int)
df['catering'] = (df['catering'] =='Есть').astype(int)
df['banquets'] = (df['banquets'] =='Есть').astype(int)

sites=list(df['site'].values)
sites1=[]
for i in range(len(sites)):
    if sites[i].startswith('http://'):
        sites1+=[1]
    else:
        sites1+=[0]       

df['site'] = sites1

telephones=list(df['telephone'].values)
telephone1=[]
for i in range(len(telephones)):
    if telephones[i].startswith('+7'):
        telephone1+=[1]
    else:
        telephone1+=[0]   
df['telephone'] = telephone1     

#df.loc[df['avg_check'] =='Нет', 'avg_check'] = 0   

avg = list(df['avg_check'].values)
avg1=[]
for i in range(len(avg)):
    if avg[i].startswith('До 7'):
        avg1+=[1]
    elif avg[i].startswith('70'):
        avg1+=[2]
    elif avg[i].startswith('170'):
        avg1+=[3]
    elif avg[i].startswith('Бо'):
        avg1+=[4]        
    else:
        avg1+=[0]   
df['avg_check'] = avg1
metros = list(df['metro'].values)
metro1=[]
metro2=[]
for i in metros:
    metro1 += [i[2:len(i)-2].split(',')]
for i in range(len(metro1)):
    if metro1[i][0] in station_metro:
        metro2 +=[1]
    else:
        metro2 +=[0] 
df['city_center']=metro2        